In [1]:
test_semantic_model_url = 'https://raw.githubusercontent.com/djouallah/Fabric_Notebooks_Demo/refs/heads/main/SemanticModel/sm/semantic_model.txt' 
SF = 0.1
output_dir="/tmp/llm"

In [2]:
import requests
import json
import duckdb
import pathlib
import time
import pandas as pd
import re
from   openai import AzureOpenAI, OpenAIError
import datetime

In [3]:
if SF <1 :
 schema = f"{str(SF).replace('.', '_')}"
else :
 schema = f'DS{SF:02d}'
import os
folder_path = '/some_location'
os.makedirs(folder_path, exist_ok=True)
db_path = folder_path +"/"+ schema +".duckdb"
if not pathlib.Path(db_path).exists():
    con = duckdb.connect(db_path)
    con.sql("SET memory_limit = '14GB' ")
    con.sql(f"CALL dsdgen(sf={SF})")
    con.close()
con = duckdb.connect()
con.sql(f""" attach or replace '{db_path}' as ds(read_only) ; use ds """)

In [4]:
def get_ai_response(user_message, LLM, provider, api_key=None, azure_endpoint=None, azure_api_version=None):
    try:
        github_response = requests.get(test_semantic_model_url)
        github_response.raise_for_status()
        system_prompt = github_response.text.strip()
    except requests.RequestException as e:
        return f"Error fetching system prompt from GitHub: {e}"

    generated_text = None

    if provider == "ollama":
        url = 'http://127.0.0.1:11434/api/chat'
        headers = {'Content-Type': 'application/json'}
        payload = {
            'model': LLM,
            'messages': [
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': user_message}
            ],
            'stream': False
        }
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            data = response.json()
            generated_text = data.get('message', {}).get('content', '')
            if generated_text:
                generated_text = generated_text.replace("<think>", "").replace("</think>", "")
        except requests.RequestException as e:
            return f"Error with Ollama API request: {e}"
        except (KeyError, IndexError, AttributeError) as e:
            return f"Unexpected response format from Ollama: {e}\nFull response data: {data}"

    elif provider == "azure_foundry":
        if not azure_endpoint or not azure_api_version:
            return "Azure provider requires 'azure_endpoint' and 'azure_api_version' parameters."

        try:
            client = AzureOpenAI(
                api_version=azure_api_version,
                azure_endpoint=azure_endpoint,
                api_key=api_key,
            )

            response = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message}
                ],
                model=LLM,
            )

            generated_text = response.choices[0].message.content

        except OpenAIError as e:
            return f"Azure OpenAI API Error: {e}"
        except Exception as e:
            return f"An unexpected error occurred with Azure OpenAI: {e}"

    else:
        return f"Unsupported provider for this test setup: {provider}. Supported providers are 'ollama' and 'azure_foundry'."

    if generated_text is None:
        return f"Could not retrieve generated text from {provider} response."

    # --- Begin SQL/Markdown Cleaning ---
    cleaned_text = re.sub(
        r'```(sql|duckdb)?\s*([\s\S]*?)\s*```',
        r'\2',
        generated_text,
        flags=re.IGNORECASE
    ).strip()

    if "SELECT " in cleaned_text.upper() or "WITH " in cleaned_text.upper() or "```" in cleaned_text:
        cleaned_text = cleaned_text.replace("```sql", "").replace("```duckdb", "").replace("```", "").strip()

    return cleaned_text


In [5]:
def execute_sql_with_retry(query, test_model,provider, api_key, azure_endpoint, azure_api_version,max_attempts):
    attempt = 1
    current_query = query.strip()

    while attempt <= max_attempts:
        try:
            # Execute the SQL query
            result = con.sql(current_query).df()
            # Return the DataFrame result and the number of attempts
            return result, attempt
        except Exception as e:
            # Catch any exception during execution
            error_message = str(e).lower()

            # Check if the error is likely a syntax, parser, or binder error
            if "syntax" not in error_message and "parser" not in error_message and "binder" not in error_message:
                # If it's a non-syntax error, return the error message and attempts
                return f"Non-syntax error: {e}", attempt

            # If it's a syntax-like error, print attempt info
            print(current_query)
            print(f"Attempt {attempt}/{max_attempts} failed with syntax error: {e}")

            # If max attempts reached, return the final error message and attempts
            if attempt == max_attempts:
                return f"Max attempts reached. Last error: {e}", attempt

            # Prepare message for Gemini to fix the query
            message = (
                f"The following SQL query has a syntax error: '{current_query}'.\n"
                f"Error message: {e}\n"
                f"Please provide the corrected SQL query. Return only the corrected query without explanation."
            )

            # Get corrected query from Gemini
            corrected_query = get_ai_response(message, test_model,provider, api_key, azure_endpoint, azure_api_version)

            # If Gemini returns an error, return that error message and attempts
            if corrected_query.startswith("Error"):
                return f"Failed to get corrected query : {corrected_query}", attempt

            #print(f"Received corrected query")
            # Update current_query with the corrected one and increment attempt
            current_query = corrected_query.strip()
            attempt += 1
    return "Unexpected error or loop termination", attempt



In [6]:
def ask_question(questions, test_model,provider, azure_endpoint=None, azure_api_version=None, api_key=None):
    results_data = [] 
    for i, x in enumerate(questions):
        print(f"Question {i+1}: {x}") # Keep or remove print as needed
        start_time = time.time()
        sql_query_or_error = get_ai_response(x, test_model, provider, api_key, azure_endpoint, azure_api_version)
        print(sql_query_or_error) # Keep or remove print as needed
        query_result_data_json = [] # Initialize as an empty list for JSON result
        attempts_count = None
        error_details = None # Initialize error_details
        if sql_query_or_error is None or sql_query_or_error.startswith("Error"):
            # If get_ai_response returned an error or None, store the error string
            error_message = sql_query_or_error if sql_query_or_error is not None else "AI response was None"
            # print(f"Failed to get query from AI: {error_message}") # Keep or remove print
            # print("Execution: SKIPPED (AI error)") # Keep or remove print
            error_details = f"AI Error: {error_message}" # Store error details
            # query_result_data_json remains empty []
            result_row_count = 0 # Result count is 0 on error
        else:
            # Attempt to execute the SQL query
            # Ensure execute_sql_with_retry returns a DataFrame on success or a string error
            # Pass the test_model to execute_sql_with_retry as it might need it
            result_from_execution, attempts_count = execute_sql_with_retry(sql_query_or_error, test_model,provider,api_key, azure_endpoint, azure_api_version, max_attempts=3)

            display(result_from_execution)
            is_successful = isinstance(result_from_execution, pd.DataFrame)

            if is_successful:
                print("Execution: SUCCESS") # Keep or remove print
                query_result_data_json = result_from_execution.to_dict('records')
                error_details = None # No error details on success
                result_row_count = len(result_from_execution) # Calculate row count
            else:
                print("Execution: FAILED") # Keep or remove print
                #query_result_data_json remains empty []
                error_details = f"Execution Error: {result_from_execution}" # Store error details
                result_row_count = 0 # Result count is 0 on failure


        end_time = time.time()
        duration = round(end_time - start_time, 2)
        print(f"\nExecution Time: {duration:.2f} seconds") # Keep or remove print
        print(f" ############################### ") # Keep or remove print
        results_data.append({
            "model" : test_model,
            "nbr": i + 1,
            "question": x,
            "duration_s": duration, # Changed key name slightly for JSON convention
            "sql_query": sql_query_or_error, # Store the generated query string (could be AI error)
            "attempts": attempts_count,
            "result": query_result_data_json, # This is now a list of dictionaries
            "result_count": result_row_count, # Use the calculated row count
            "error_details": error_details # Store error details
        })

    os.makedirs(output_dir, exist_ok=True)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"results_{timestamp}.json"
    output_path = os.path.join(output_dir, output_filename)
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results_data, f, indent=4)
        return f"Successfully processed {len(questions)} questions. Results saved to {output_path}"
    except IOError as e:
        return f"Error saving results to {output_path}: {e}"
    except Exception as e:
        return f"An unexpected error occurred during file saving: {e}"

In [7]:
questions = [
    # Easy Questions:
    "What is the overall total sales revenue?",
    "What is the total number of items sold across all transactions?",
    "What is the total monetary value of all returned items?",
    "List the names of all stores.",
    "What are the different item categories available?",

    # Medium Questions:
    "Show total sales revenue for each year.",
    "Which store generated the most total sales revenue?",
    "What is the total quantity of items sold, broken down by item brand?",
    "Compare total sales from preferred customers versus non-preferred customers.",
    "What is the total return amount for each city where stores are located?",

    # Hard Questions:
    "What is the net sales for each store name?",
    "Calculate the return rate for each item category.",
    "What is the monthly trend of net sales during the year 2001?",
    "Which customer birth country exhibits the highest average return rate?",
    "List all item product names that have a return rate greater than 5% and their total sales.",

    # Very Hard Questions:
    "For each store, what was the percentage change in net sales between two consecutive recent years (e.g., 2001 and 2002)?",
    "What is the return rate for items sold on weekends versus weekdays, broken down by customer age groups (e.g., under 30, 30-45, over 45)?",
    "Which item brand has shown the largest decrease in its return rate when comparing the average rate of 2001 to 2002, specifically for stores located in the 'TN' state?",
    "For each item class, compare the average net sales value per sales transaction between preferred and non-preferred customers.",
    "Identify stores whose net sales are below 99% of the average net sales for stores in their respective countries, and list their manager, city, and current overall return rate."
]

In [8]:
%%time
ask_question(questions,'qwen3:4b',"ollama")

Question 1: What is the overall total sales revenue?
SELECT
  total_sales
FROM (
  SELECT
    SUM(ss.ss_sales_price * ss.ss_quantity) AS total_sales
  FROM store_sales AS ss
) AS sales_agg; -- overall total sales revenue


,total_sales
0,5.143009e+08


Execution: SUCCESS

Execution Time: 15.74 seconds
 ############################### 
Question 2: What is the total number of items sold across all transactions?
SELECT
  SUM(ss.ss_quantity) AS total_quantity
FROM
  store_sales ss; -- total_quantity measure


,total_quantity
0,13939520.0


Execution: SUCCESS

Execution Time: 2.13 seconds
 ############################### 
Question 3: What is the total monetary value of all returned items?
-- total monetary value of all returned items
SELECT
  SUM(sr.sr_return_amt) AS total_returns
FROM store_returns sr;


...

Execution: SUCCESS

Execution Time: 12.30 seconds
 ############################### 
Question 20: Identify stores whose net sales are below 99% of the average net sales for stores in their respective countries, and list their manager, city, and current overall return rate.
SELECT
  s.s_store_name,
  s.s_manager,
  s.s_city,
  (COALESCE(r.total_returns, 0) / NULLIF(COALESCE(s.total_sales, 0), 0)) * 100 AS return_rate
FROM (
  SELECT
    st.s_store_sk,
    st.s_store_name,
    st.s_manager,
    st.s_city,
    SUM(ss.ss_sales_price * ss.ss_quantity) AS total_sales
  FROM store_sales AS ss
  INNER JOIN store AS st ON ss.ss_store_sk = st.s_store_sk
  GROUP BY st.s_store_sk, st.s_store_name, st.s_manager, st.s_city
) s
LEFT JOIN (
  SELECT
    st.s_store_sk,
    SUM(sr.sr_return_amt) AS total_returns
  FROM store_returns AS sr
  INNER JOIN store AS st ON sr.sr_store_sk = st.s_store_sk
  GROUP BY st.s_store_sk
) r ON s.s_store_sk = r.s_store_sk
LEFT JOIN (
  SELECT
    s_store_sk,
    AVG(tota

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,s_store_name,s_manager,s_city,return_rate
0,ought,William Ward,Midway,5.148919


Execution: SUCCESS

Execution Time: 151.95 seconds
 ############################### 
CPU times: total: 15min 53s
Wall time: 17min 37s


'Successfully processed 20 questions. Results saved to /tmp/llm\\results_20250508_223200.json'

In [9]:
%%time
ask_question(questions,"gpt-4o","azure_foundry", os.getenv("llm_endpoint"),"2025-01-01-preview" ,os.getenv("LLM_API_KEY"))

Question 1: What is the overall total sales revenue?
-- What is the overall total sales revenue?
SELECT 
    SUM(ss.ss_sales_price * ss.ss_quantity) AS total_sales
FROM store_sales AS ss;


,total_sales
0,5.143009e+08


Execution: SUCCESS

Execution Time: 1.78 seconds
 ############################### 
Question 2: What is the total number of items sold across all transactions?
-- What is the total number of items sold across all transactions?
SELECT
    SUM(ss.ss_quantity) AS total_items_sold
FROM store_sales AS ss;


,total_items_sold
0,13939520.0


Execution: SUCCESS

Execution Time: 2.15 seconds
 ############################### 
Question 3: What is the total monetary value of all returned items?
-- What is the total monetary value of all returned items?
SELECT
    SUM(sr.sr_return_amt) AS total_returns
FROM store_returns AS sr;


...

,item_class,preferred_customer,avg_net_sales_per_transaction
0,accent,N,1641.040888
1,accent,Y,1801.508211
2,accent,None,1879.990278
3,accent,None,NaN
4,accessories,N,1790.163312
...,...,...,...
396,None,N,NaN
397,None,Y,1900.125521
398,None,Y,NaN
399,None,None,2081.591389


Execution: SUCCESS

Execution Time: 12.91 seconds
 ############################### 
Question 20: Identify stores whose net sales are below 99% of the average net sales for stores in their respective countries, and list their manager, city, and current overall return rate.
-- Identify stores whose net sales are below 99% of the average net sales for stores in their respective countries, and list their manager, city, and current overall return rate.

WITH store_sales_agg AS (
    SELECT
        st.s_store_sk,
        st.s_country,
        st.s_store_name,
        SUM(ss.ss_sales_price * ss.ss_quantity) AS total_sales
    FROM store_sales AS ss
    INNER JOIN store AS st ON ss.ss_store_sk = st.s_store_sk
    GROUP BY st.s_store_sk, st.s_country, st.s_store_name
), store_returns_agg AS (
    SELECT
        st.s_store_sk,
        SUM(sr.sr_return_amt) AS total_returns
    FROM store_returns AS sr
    INNER JOIN store AS st ON sr.sr_store_sk = st.s_store_sk
    GROUP BY st.s_store_sk
), stor

,store_manager,city,overall_return_rate


Execution: SUCCESS

Execution Time: 8.02 seconds
 ############################### 
CPU times: total: 17.8 s
Wall time: 2min 5s


'Successfully processed 20 questions. Results saved to /tmp/llm\\results_20250508_223405.json'

In [12]:
duckdb.sql(f""" from '{output_dir}/*.json' order by nbr,model """).to_view("results")

┌──────────┬───────┬──────────────────────┬────────────┬──────────────────────┬──────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [11]:
duckdb.sql(""" from results where error_details is not null""")

┌──────────┬───────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────